In [ ]:
import os
import time
import math
import random
import numpy as np
import pandas as pd
from google import genai
from google.genai import types
from dotenv import load_dotenv

In [ ]:
random.seed(42)
load_dotenv()
client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))
prompt_template = ''

In [3]:
alay_dict = pd.read_csv('./new_kamusalay.csv', encoding='latin-1', header=None)
alay_dict = alay_dict.rename(columns={0: 'original', 
                                    1: 'replacement'})

def lowercase(text):
    return text.lower()

def remove_unnecessary_char(text):
    text = re.sub('\n',' ',text) # Remove every '\n'
    text = re.sub('rt',' ',text) # Remove every retweet symbol
    text = re.sub('user',' ',text) # Remove every username
    text = re.sub('url', ' ', text) # Remove every URL
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))',' ',text) # Remove every URL
    text = re.sub(r'\b(?:x[a-fA-F0-9]{2}\s*)+\b', '', text) # Remove emoji bytecode
    text = re.sub('  +', ' ', text) # Remove extra spaces
    return text
    
def remove_nonaplhanumeric(text):
    text = re.sub('[^0-9a-zA-Z]+', ' ', text) 
    return text

def normalize_alay(text):
    alay_dict_map = dict(zip(alay_dict['original'], alay_dict['replacement']))
    return ' '.join([alay_dict_map[word] if word in alay_dict_map else word for word in text.split(' ')])

print("remove_nonaplhanumeric: ", remove_nonaplhanumeric("Halooo,,,,, duniaa \x8f \xd2\1 !!"))
print("lowercase: ", lowercase("Halooo, duniaa!"))
print("remove_unnecessary_char: ", remove_unnecessary_char("Hehe\n\n RT USER USER apa kabs www.google.com\n  hehe URL xf8 x2a x89"))
print("normalize_alay: ", normalize_alay("aamiin adek abis"))

remove_nonaplhanumeric:  Halooo duniaa 
lowercase:  halooo, duniaa!
remove_unnecessary_char:  Hehe RT USER USER apa kabs hehe URL 
normalize_alay:  amin adik habis


In [4]:
def preprocess(text):
    text = lowercase(text)
    text = remove_nonaplhanumeric(text)
    text = remove_unnecessary_char(text)
    text = normalize_alay(text) 

    return text

In [5]:
def load_data():
    data = pd.read_csv('./val_data.csv', encoding='latin-1')
    data['Tweet'] = data['Tweet'].apply(preprocess)
    
    return data

In [6]:
def build_prompt(text):
    return prompt_template.replace('KALIMAT:', text)

In [ ]:
safety_settings = [
    types.SafetySetting(
        category=types.HarmCategory.HARM_CATEGORY_HATE_SPEECH,
        threshold=types.HarmBlockThreshold.BLOCK_NONE,
    ),
    types.SafetySetting(
        category=types.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
        threshold=types.HarmBlockThreshold.BLOCK_NONE,
    ),
    types.SafetySetting(
        category=types.HarmCategory.HARM_CATEGORY_HARASSMENT,
        threshold=types.HarmBlockThreshold.BLOCK_NONE,
    ),
    types.SafetySetting(
        category=types.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
        threshold=types.HarmBlockThreshold.BLOCK_NONE,
    ),
    types.SafetySetting(
        category=types.HarmCategory.HARM_CATEGORY_CIVIC_INTEGRITY,
        threshold=types.HarmBlockThreshold.BLOCK_NONE,
    ),
]

def generate_noise(prompt):
    response = client.models.generate_content(
        model="gemini-2.5-flash", 
        contents=prompt, 
        config=types.GenerateContentConfig(
            safety_settings=safety_settings,
        )
    )
    return response.text

# CHAR LEVEL NOISE

In [25]:
# Typos
prompt_template = f'''KALIMAT:

- lakukan noise injection dengan mensimulasikan typo pada kalimat tersebut.
- lakukan pada maksimal 25% dari kata yang ada pada kalimat tersebut.
- output hanya berupa kalimat akhir yang telah diubah
- jangan tambahkan markdown dan tanda baca apapun
- gunakan huruf kecil'''

In [26]:
data = load_data()

In [27]:
augmented_cnt = 0
for i in range(len(data)):
    n = random.uniform(0, 1)

    if n <= 0.1:
        augmented_cnt += 1
        text = data.iloc[i]['Tweet']

        print("Index:", i)
        print("Original:", text, '\n')

        augmented = generate_noise(build_prompt(text))
        data["Tweet"].values[i] = augmented

        print("Augmented:", augmented)
        print("=" * 50)

        time.sleep(4)

Index: 1
Original:  jangan lemah pak polisi hukum saja memilih di bui apa kerja sosial memandikan singa apa buaya di kebun binatang  

Augmented: jangan lemah pak plisi hukuk saja memlih di bui apa kerja sosial memandikan singa apa buaya di kebn binatang
Index: 7
Original:  alah pak pak yang gaduh itu kan cuma orang orang surat pemberitahuan tahunan bapak sekarang ya tenang lah lapak sudah dikuasai tinggal bagi bagi kue ke sesama gerombolan kemarin kan pada kelaparan tidak dapat apa pun mau maling pun susah rakyat makin parah  

Augmented: alah pak pak yang gdauh itu kan cumaa orang orang surat pemberitahuan thunan bapak sekarang ya tenang lah laak sudah dikasai tingal bagi bagi kue ke sesama gerombolan kemarin kan pada kelapaarn tidak dapat apa pun mau mling pun sush rakyat makin parah
Index: 9
Original: pengusaha cina jika reklamasi pulau tidak dibatalkan jokowi akan kami lengserkan 

Augmented: pengusaha cnina jika reklamasi pulau tidak dibatalkan jokow akan kami lengserkan
Index: 1

In [28]:
print("Total noisy data generated:", augmented_cnt)

Total noisy data generated: 248


In [29]:
data.to_csv('./char-level-noise.csv', index=False)

# SYNONYM REPLACEMENT

In [ ]:
prompt_template = f'''KALIMAT:

- lakukan noise injection dengan mengganti maksimum 25% kata dalam kalimat dengan sinonim nya
- output hanya berupa 1 baris kalimat akhir yang telah diubah
- jangan tambahkan markdown dan tanda baca apapun
- jangan hilangkan kata selain dari kata yang diganti
- gunakan huruf kecil'''

In [232]:
data = load_data()
data.head()

,original_text,source,pornografi,sara,radikalisme,pencemaran_nama_baik,processed_text
0,Kalo urusan THR kompak yee. Gak ada kafir kafi...,twitter,0,1,0,0,kalau urusan thr kompak ya tidak ada kafir kaf...
1,[url=&quot;https://m.republika.co.id/amp_versi...,kaskus,0,1,0,0,url menang berkat dua pemain muslim dan url en...
2,Pantatnya ga sempurna banyak lemak Haha.. Kata...,instagram,1,0,0,1,pantatnya tidak sempurna banyak lemak tertawa ...
3,\n\nSalut buat para saudara saudara ku yang be...,kaskus,0,0,1,0,dan nsalut buat para saudara saudara ku yang b...
4,"Yah lu min,, lu mau goblokin org,, tpi lu mala...",instagram,0,0,0,1,yah kamu administrator kamu mau goblokin orang...


In [ ]:
random.seed(42)
augmented_cnt = 0

for i in range(len(data)):
    n = random.uniform(0, 1)

    if n <= 0.1:
        augmented_cnt += 1
        text = data.iloc[i]['Tweet']

        print("Index:", i)
        print("Original:", text, '\n')

        augmented = generate_noise(build_prompt(text))
        data["Tweet"].values[i] = augmented

        print("Augmented:", augmented)
        print("=" * 50)
        time.sleep(4)

Index: 1
Original:  jangan lemah pak polisi hukum saja memilih di bui apa kerja sosial memandikan singa apa buaya di kebun binatang  

Augmented: jangan ragu pak aparat hukum saja memilih di penjara apa tugas sosial memandikan singa apa buaya di kebun binatang
Index: 7
Original:  alah pak pak yang gaduh itu kan cuma orang orang surat pemberitahuan tahunan bapak sekarang ya tenang lah lapak sudah dikuasai tinggal bagi bagi kue ke sesama gerombolan kemarin kan pada kelaparan tidak dapat apa pun mau maling pun susah rakyat makin parah  

Augmented: alah pak pak yang ribut itu kan hanya oknum oknum surat pemberitahuan tahunan bapak kini ya santai lah wilayah sudah dikendalikan tinggal bagi bagi keuntungan ke sesama kelompok kemarin kan pada kelaparan tidak dapat apa pun mau maling pun sulit rakyat makin parah
Index: 9
Original: pengusaha cina jika reklamasi pulau tidak dibatalkan jokowi akan kami lengserkan 

Augmented: pebisnis cina kalau reklamasi pulau tidak dibatalkan jokowi akan kami 

In [ ]:
print("Total noisy data generated:", augmented_cnt)

Total noisy data generated: 248


In [ ]:
data.to_csv('./noisy/synonym-replacement.csv', index=False)

# WORD INSERTION

In [ ]:
prompt_template = f'''KALIMAT:

- lakukan noise injection dengan menyisipkan kata-kata irrelevan yang membuat kalimat menjadi tidak nyambung
- sebar 3-6 kata tambahan di seluruh kalimat
- jangan mengubah kata yang sudah ada, hanya sisipkan kata baru
- output hanya berupa 1 baris kalimat akhir yang telah diubah
- jangan tambahkan markdown dan tanda baca apapun
- jangan hilangkan kata selain dari kata yang diganti
- gunakan huruf kecil'''

In [45]:
data = load_data()
data.head()

,Tweet,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other,HS_Weak,HS_Moderate,HS_Strong
0,gue merasa gue tidak sipit mata gue kecil saja...,0,0,0,0,0,0,0,0,0,0,0,0
1,jangan lemah pak polisi hukum saja memilih di...,0,0,0,0,0,0,0,0,0,0,0,0
2,banyak orang tionghoa pinggiran yang baik baik...,0,0,0,0,0,0,0,0,0,0,0,0
3,nikmat banget duo cowok ganteng habis lagi ng...,0,1,0,0,0,0,0,0,0,0,0,0
4,hoaks ala pak yang mampu menyihir dua ratus li...,1,0,1,0,0,0,0,0,1,1,0,0


In [47]:
random.seed(42)
augmented_cnt = 0

for i in range(len(data)):
    n = random.uniform(0, 1)

    if n <= 0.1:
        augmented_cnt += 1
        text = data.iloc[i]['Tweet']

        print("Index:", i)
        print("Original:", text, '\n')

        augmented = generate_noise(build_prompt(text))
        data["Tweet"].values[i] = augmented

        print("Augmented:", augmented)
        print("=" * 50)

        time.sleep(4)

Index: 1
Original: jangan tiba-tiba lemah pak polisi sebenarnya hukum saja lalu memilih di bui apa kerja sosial memandikan singa tentu saja apa buaya di kebun binatang 

Augmented: jangan kucing tiba-tiba lemah pak polisi terbang sebenarnya hukum saja lalu nasi memilih di bui apa kerja sosial memandikan singa mobil tentu saja apa buaya di kebun binatang biru
Index: 7
Original:  alah pak pak yang gaduh itu kan cuma orang orang surat pemberitahuan tahunan bapak sekarang ya tenang lah lapak sudah dikuasai tinggal bagi bagi kue ke sesama gerombolan kemarin kan pada kelaparan tidak dapat apa pun mau maling pun susah rakyat makin parah  

Augmented: alah meja pak pak yang biru gaduh itu kan cuma orang orang lompat surat pemberitahuan tahunan bapak sekarang ya bunga tenang lah lapak sudah dikuasai tinggal bagi bagi kue ke diam sesama gerombolan kemarin kan pada cepat kelaparan tidak dapat apa pun mau maling pun susah rakyat makin asin parah
Index: 9
Original: pengusaha cina jika reklamasi pul

In [48]:
print("Total noisy data generated:", augmented_cnt)

Total noisy data generated: 248


In [49]:
data.to_csv('./noisy/word-insertion.csv', index=False)

# WORD DELETION

In [18]:
data = load_data()
data.head()

,Tweet,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other,HS_Weak,HS_Moderate,HS_Strong
0,gue merasa gue tidak sipit mata gue kecil saja...,0,0,0,0,0,0,0,0,0,0,0,0
1,jangan lemah pak polisi hukum saja memilih di...,0,0,0,0,0,0,0,0,0,0,0,0
2,banyak orang tionghoa pinggiran yang baik baik...,0,0,0,0,0,0,0,0,0,0,0,0
3,nikmat banget duo cowok ganteng habis lagi ng...,0,1,0,0,0,0,0,0,0,0,0,0
4,hoaks ala pak yang mampu menyihir dua ratus li...,1,0,1,0,0,0,0,0,1,1,0,0


In [19]:
random.seed(42)
augmented_cnt = 0
for i in range(len(data)):
    n = random.uniform(0, 1)

    if n <= 0.1:
        augmented_cnt += 1
        text = data.iloc[i]['Tweet']
        words = text.split()
        num_words = len(words)
        num_to_del = math.ceil(len(text.split(' ')) / 4)

        print("Index:", i)
        print("Original:", text, '\n')

        if num_to_del > 0:
            indices_to_delete = np.random.choice(num_words, size=num_to_del, replace=False)
            mask = np.ones(num_words, dtype=bool)
            mask[indices_to_delete] = False
            augmented_words = np.array(words)[mask]
            augmented = ' '.join(augmented_words)
        else:
            augmented = text  # if too short, don't change

        data.at[i, "Tweet"] = augmented

        print("Augmented:", augmented)
        print("=" * 50)
        # time.sleep(5)

Index: 1
Original:  jangan lemah pak polisi hukum saja memilih di bui apa kerja sosial memandikan singa apa buaya di kebun binatang  

Augmented: jangan lemah polisi hukum saja bui apa sosial memandikan singa apa buaya kebun
Index: 7
Original:  alah pak pak yang gaduh itu kan cuma orang orang surat pemberitahuan tahunan bapak sekarang ya tenang lah lapak sudah dikuasai tinggal bagi bagi kue ke sesama gerombolan kemarin kan pada kelaparan tidak dapat apa pun mau maling pun susah rakyat makin parah  

Augmented: alah yang itu kan orang orang tahunan bapak sekarang ya tenang lah lapak tinggal bagi kue ke sesama kemarin kan pada tidak dapat apa pun mau maling pun rakyat makin parah
Index: 9
Original: pengusaha cina jika reklamasi pulau tidak dibatalkan jokowi akan kami lengserkan 

Augmented: pengusaha jika pulau dibatalkan jokowi akan kami lengserkan
Index: 12
Original: usir dan depak cina dan amerika komunisme sekulerisme liberalisme dan demokrasi ganti dengan hukum allah dan rasulnya gu

In [20]:
print("Total noisy data generated:", augmented_cnt)

Total noisy data generated: 248


In [21]:
data.to_csv('./noisy/word-deletion.csv', index=False)

# COMBINED

In [ ]:
prompt_template = f'''KALIMAT:

- lakukan 3 noise injection: typo, penghapusan kata, atau penyisipan kata-kata irrelevan yang membuat kalimat menjadi tidak nyambung
- lakukan noise injection pada maksimum 40% kata dari kalimat
- pastikan kalimat akhir memiliki masing-masing macam noise injection
- jika ada beberapa kata yang sama, hanya ubah satu kata saja
- jangan tambahkan markdown dan tanda baca apapun
- gunakan huruf kecil
- output hanya berupa kalimat akhir yang telah diubah
'''

In [9]:
data = load_data()
data.head()

,Tweet,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other,HS_Weak,HS_Moderate,HS_Strong
0,gue merasa gue tidak sipit mata gue kecil saja...,0,0,0,0,0,0,0,0,0,0,0,0
1,jangan lemah pak polisi hukum saja memilih di...,0,0,0,0,0,0,0,0,0,0,0,0
2,banyak orang tionghoa pinggiran yang baik baik...,0,0,0,0,0,0,0,0,0,0,0,0
3,nikmat banget duo cowok ganteng habis lagi ng...,0,1,0,0,0,0,0,0,0,0,0,0
4,hoaks ala pak yang mampu menyihir dua ratus li...,1,0,1,0,0,0,0,0,1,1,0,0


In [10]:
random.seed(42)
augmented_cnt = 0

for i in range(len(data)):
    n = random.uniform(0, 1)

    if n <= 0.1:
        augmented_cnt += 1
        text = data.iloc[i]['Tweet']

        print("Index:", i)
        print("Original:", text, '\n')

        augmented = generate_noise(build_prompt(text))
        data["Tweet"].values[i] = augmented

        print("Augmented:", augmented)
        print("=" * 50)

        time.sleep(3.5)

Index: 1
Original:  jangan lemah pak polisi hukum saja memilih di bui apa kerja sosial memandikan singa apa buaya di kebun binatang  

Augmented: jangan lemah pak polisi hokum memilih nasi rumah di bui apa kerja sosial memandikan singa apa buaya di kebun binatang
Index: 7
Original:  alah pak pak yang gaduh itu kan cuma orang orang surat pemberitahuan tahunan bapak sekarang ya tenang lah lapak sudah dikuasai tinggal bagi bagi kue ke sesama gerombolan kemarin kan pada kelaparan tidak dapat apa pun mau maling pun susah rakyat makin parah  

Augmented: alah pak yang gaduh kan orang pemebritahuan bapak ini sekarang tenang lapak hijau sudah dikuasai bagi bagi sesama geromobolan kemarin kan pada kelaparan sembunyi dapat apa mau maling pun susah rakyat makin paraah
Index: 9
Original: pengusaha cina jika reklamasi pulau tidak dibatalkan jokowi akan kami lengserkan 

Augmented: pengusah cina jika reklamasi pulau pisang dibatalkan jokowi akan kami lengserkan
Index: 12
Original: usir dan depak cin

In [11]:
print("Total noisy data generated:", augmented_cnt)

Total noisy data generated: 248


In [12]:
data.to_csv('./noisy/combined-noise.csv', index=False)